In [1]:
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from linevul_model import Model
import torch
import numpy

In [2]:
device = 'cpu'
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
config.num_attention_heads = 12
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', config=config, ignore_mismatched_sizes=True)
model = Model(model, config, tokenizer, []).to(device)
state_dict = torch.load('./saved_models/checkpoint-best-f1/endor-model.bin', map_location=device)
model.load_state_dict(state_dict, strict=False)
model.to(device)
''''''


Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be 

''

In [3]:
texts = ['''
public String getType() {
  if (type == null && resource.hasProperty(DCTerms.type)) {
    type = resource.getPropertyResourceValue(DCTerms.type).getURI();
  }
  return type;
}
''', '''
public String getSelectedText() {
  if (selectedText == null && resource.hasProperty(FISE.SELECTED_TEXT)) {
    selectedText = resource.getProperty(FISE.SELECTED_TEXT).getString();
  }
  return selectedText;
}
''', '''
public String getSelectionContext() {
  if (selectionContext == null && resource.hasProperty(FISE.SELECTION_CONTEXT)) {
    selectionContext = resource.getProperty(FISE.SELECTION_CONTEXT).getString();
  }
  return selectionContext;
}
''', '''
public Long getStart() {
  if (start == null && resource.hasProperty(FISE.START)) {
    start = resource.getProperty(FISE.START).getLong();
  }
  return start;
}
''', '''
public Long getEnd() {
  if (end == null && resource.hasProperty(FISE.END)) {
    end = resource.getProperty(FISE.END).getLong();
  }
  return end;
}
''', '''

''']

In [7]:
for text in texts:
    input_ids = tokenizer.encode(text, return_tensors="pt")
    input_ids = input_ids.to(device)
    model.eval()
    with torch.no_grad():
        logits = model(input_ids=input_ids)
    logits = numpy.array(logits)
    print(int(logits[:,1]))


0
0
0
0
0
